### Langchain Model

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGSMITH_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")


from langchain_ollama import ChatOllama

# Create model
ollama_model = ChatOllama(model="llama3.1")

ollama_model

### Get the response from llm

In [ ]:
response = ollama_model.invoke("Write 5 line on the Lion.")

response

In [ ]:
response.content

### Streaming response

In [ ]:
response = ollama_model.stream("write 10 lines on the Lion.") # response it the List of AIMessage
for chunk in response:
    print(chunk.text, end="|", flush=True)

### Batch processing

In [ ]:
inputList = ["5 lines on Lion", "5 lines on Tiger", "5 line on Elephant"]
responses = ollama_model.batch(inputs=inputList)
for response in responses:
    print(response)

In [ ]:
inputList = ["5 lines on Lion", "5 lines on Tiger", "5 line on Elephant"]
responses = ollama_model.batch_as_completed(inputs=inputList)
for response in responses:
    print(response)

### Batch processing with configurations

In [ ]:
inputList = ["5 lines on Lion", "5 lines on Tiger", "5 line on Elephant"]
responses = ollama_model.batch_as_completed(inputs=inputList, config={
    'max_concurrency': 1
})
for response in responses:
    print(response)

### Register a tool

In [ ]:
# Define tools
from langchain.tools import tool
from typing import List, Union

@tool
def get_weather(location: Union[str, List[str]] = None, locations: Union[str, List[str]] = None) -> str:
    """Get the weather for a location or list of locations.
    
    Args:
        location: A single location or list of locations.
        locations: Alias for location, often used by models when multiple locations are requested.
    """
    # Handle argument alias
    final_locs = location if location is not None else locations
    
    if final_locs is None:
        return "Please provide a location."

    if isinstance(final_locs, list):
        return "\n".join([f"The weather in {loc} is sunny." for loc in final_locs])
    return f"The weather in {final_locs} is sunny."

# bind tools to the model
model_with_tools = ollama_model.bind_tools([get_weather])

### Tool calling - execution loop

In [ ]:

# Model generates tool calls
messages = [
    {"role": "user", "content": "What is the weather in New York, Bangalore and Noida?"}
]
ai_messages = model_with_tools.invoke(messages)
messages.append(ai_messages)

# Invoke tools and collect responses
for tool_call in ai_messages.tool_calls:    
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# model generate final response
ai_messages = model_with_tools.invoke(messages)
print(ai_messages.content)
